Copyright Jana Schaich Borg/Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)

# MySQL Exercise 11: Queries that Test Relationships Between Test Completion and Dog Characteristics



## 1. Assess whether Dognition personality dimensions are related to the number of tests completed 


**Question 1: To get a feeling for what kind of values exist in the Dognition personality dimension column, write a query that will output all of the distinct values in the dimension column.  Use your relational schema or the course materials to determine what table the dimension column is in. **


In [ ]:
%%sql
SELECT DISTINCT(dimension)
FROM dogs;


**Question 2: Use the equijoin syntax (described in MySQL Exercise 8) to write a query that will output the Dognition personality dimension and total number of tests completed by each unique DogID.  This query will be used as an inner subquery in the next question.  LIMIT your output to 100 rows for troubleshooting purposes.**

In [ ]:
%%sql
SELECT d.dog_guid AS DogID, d.dimension AS Dimension, COUNT(c.created_at) AS NumTests
FROM dogs d, complete_tests c
WHERE d.dog_guid=c.dog_guid
GROUP BY DogID
LIMIT 100;

**Question 3: Re-write the query in Question 2 using traditional join syntax (described in MySQL Exercise 8).**

In [ ]:
%%sql
SELECT d.dog_guid AS DogID, d.dimension AS Dimension, COUNT(c.created_at) AS NumTests
FROM dogs d JOIN complete_tests c
    ON d.dog_guid=c.dog_guid
GROUP BY DogID
LIMIT 100;

 
**Question 4: To start, write a query that will output the average number of tests completed by unique dogs in each Dognition personality dimension.  Choose either the query in Question 2 or 3 to serve as an inner query in your main query. **


In [ ]:
%%sql
SELECT dimension AS Dimension, AVG(DimensionData.NumTests) AS AvgTests
FROM (SELECT d.dog_guid AS DogID, d.dimension AS Dimension, COUNT(c.created_at) AS NumTests
        FROM dogs d, complete_tests c
        WHERE d.dog_guid=c.dog_guid
        GROUP BY DogID) AS DimensionData
GROUP BY Dimension
ORDER BY AvgTests DESC;


**Question 5: How many unique DogIDs are summarized in the Dognition dimensions labeled "None" or ""? (You should retrieve values of 13,705 and 71)**

In [ ]:
%%sql
SELECT Dimension, COUNT(DogID)
FROM (SELECT d.dog_guid AS DogID, d.dimension AS Dimension, COUNT(c.created_at) AS NumTests
        FROM dogs d, complete_tests c
        WHERE d.dog_guid=c.dog_guid AND (dimension is NULL OR dimension = "")
        GROUP BY DogID) AS DimensionData
GROUP BY Dimension;


**Question 6: To determine whether there are any features that are common to all dogs that have non-NULL empty strings in the dimension column, write a query that outputs the breed, weight, value in the "exclude" column, first or minimum time stamp in the complete_tests table, last or maximum time stamp in the complete_tests table, and total number of tests completed by each unique DogID that has a non-NULL empty string in the dimension column.**


In [ ]:
%%sql
SELECT DISTINCT(d.dog_guid) AS DogID, d.breed, d.weight, d.exclude, 
    MIN(c.created_at) AS FirstTS, MAX(c.updated_at) AS LastTS, COUNT(c.created_at) AS NumTests, d.dimension
FROM dogs d, complete_tests c
WHERE d.dog_guid = c.dog_guid AND d.dimension = ''
GROUP BY DogID;


**Question 7: Rewrite the query in Question 4 to exclude DogIDs with (1) non-NULL empty strings in the dimension column, (2) NULL values in the dimension column, and (3) values of "1" in the exclude column.  **

In [ ]:
%%sql
SELECT dimension AS Dimension, AVG(NumTests) AS NumTests, COUNT(DogID) AS NumDog
FROM (SELECT d.dog_guid AS DogID, d.dimension AS Dimension, COUNT(c.created_at) AS NumTests
        FROM dogs d, complete_tests c
        WHERE d.dog_guid=c.dog_guid AND (d.dimension is not NULL AND d.dimension != '') 
          AND (d.exclude = 0 OR d.exclude is NULL)
        GROUP BY DogID) AS DimensionData
GROUP BY Dimension
ORDER BY NumTests DESC;



## 2. Assess whether dog breeds are related to the number of tests completed

**Questions 8: Write a query that will output all of the distinct values in the breed_group field.**

In [ ]:
%%sql
SELECT DISTINCT breed_group
FROM dogs;


**Question 9: Write a query that outputs the breed, weight, value in the "exclude" column, first or minimum time stamp in the complete_tests table, last or maximum time stamp in the complete_tests table, and total number of tests completed by each unique DogID that has a NULL value in the breed_group column.**

In [ ]:
%%sql
SELECT d.breed, d.weight, d.exclude,
    MIN(c.created_at), MAX(c.created_at), COUNT(c.created_at) AS NumTest
FROM dogs d, complete_tests c
WHERE d.dog_guid=c.dog_guid AND d.breed_group is NULL
GROUP BY d.dog_guid;



**Question 10: Adapt the query in Question 7 to examine the relationship between breed_group and number of tests completed.  Exclude DogIDs with values of "1" in the exclude column. Your results should return 1774 DogIDs in the Herding breed group.**


In [ ]:
%%sql
SELECT BreedG, AVG(NumTests) AS NumTests, COUNT(DogID) AS NumDog
FROM (SELECT d.dog_guid AS DogID, d.breed_group AS BreedG, COUNT(c.created_at) AS NumTests
        FROM dogs d, complete_tests c
        WHERE d.dog_guid=c.dog_guid AND
          (d.exclude = 0 OR d.exclude is NULL)
        GROUP BY DogID) AS DimensionData
GROUP BY BreedG
ORDER BY NumTests DESC;


**Question 11: Adapt the query in Question 10 to only report results for Sporting, Hound, Herding, and Working breed_groups using an IN clause.**

In [ ]:
%%sql
SELECT BreedG, AVG(NumTests) AS NumTests, COUNT(DogID) AS NumDog
FROM (SELECT d.dog_guid AS DogID, d.breed_group AS BreedG, COUNT(c.created_at) AS NumTests
        FROM dogs d, complete_tests c
        WHERE d.dog_guid=c.dog_guid AND
          (d.exclude = 0 OR d.exclude is NULL) AND
          breed_group IN('Sporting', 'Hound', 'Herding', 'Working')
        GROUP BY DogID) AS DimensionData
GROUP BY BreedG
ORDER BY NumTests DESC;


**Questions 12: Begin by writing a query that will output all of the distinct values in the breed_type field.**

In [ ]:
%%sql
SELECT DISTINCT(breed_type)
FROM dogs;

**Question 13: Adapt the query in Question 7 to examine the relationship between breed_type and number of tests completed. Exclude DogIDs with values of "1" in the exclude column. Your results should return 8865 DogIDs in the Pure Breed group.**

In [ ]:
%%sql
SELECT BreedType, AVG(NumTests) AS NumTests, COUNT(DogID) AS NumDog
FROM (SELECT d.dog_guid AS DogID, d.breed_type AS BreedType, COUNT(c.created_at) AS NumTests
        FROM dogs d, complete_tests c
        WHERE d.dog_guid=c.dog_guid AND
          (d.exclude = 0 OR d.exclude is NULL)
        GROUP BY DogID) AS DimensionData
GROUP BY BreedType
ORDER BY NumTests DESC;

    
  
## 3. Assess whether dog breeds and neutering are related to the number of tests completed  

**Question 14: For each unique DogID, output its dog_guid, breed_type, number of completed tests, and use a CASE statement to include an extra column with a string that reads "Pure_Breed" whenever breed_type equals 'Pure Breed" and "Not_Pure_Breed" whenever breed_type equals anything else.  LIMIT your output to 50 rows for troubleshooting.**

In [ ]:
%%sql
SELECT d.dog_guid, d.breed_type, 
    CASE 
    WHEN d.breed_type='Pure Breed' THEN 'Pure_Breed'
    ELSE 'Not_Pure_Breed' 
    END AS 'Breed Type',
    COUNT(c.created_at)
FROM dogs d, complete_tests c
WHERE d.dog_guid=c.dog_guid
GROUP BY d.dog_guid
LIMIT 50;

**Question 15: Adapt your queries from Questions 7 and 14 to examine the relationship between breed_type and number of tests completed by Pure_Breed dogs and non_Pure_Breed dogs. **

In [ ]:
%%sql
SELECT BreedType, AVG(NumTests) AS NumTests, COUNT(DogID) AS NumDog
FROM (SELECT d.dog_guid AS DogID, d.breed_type, 
        CASE 
            WHEN d.breed_type='Pure Breed' THEN 'Pure_Breed'
            ELSE 'Not_Pure_Breed' 
            END AS BreedType,
      
        COUNT(c.created_at) AS NumTests
        FROM dogs d, complete_tests c
        WHERE d.dog_guid=c.dog_guid AND (d.exclude = 0 OR d.exclude is NULL)
        GROUP BY d.dog_guid) AS BreedData
GROUP BY BreedType
ORDER BY NumTests DESC;

**Question 16: Adapt your query from Question 15 to examine the relationship between breed_type, whether or not a dog was neutered (indicated in the dog_fixed field), and number of tests completed by Pure_Breed dogs and non_Pure_Breed dogs. **

In [ ]:
%%sql
SELECT BreedType, DogFix, AVG(NumTests) AS NumTests, COUNT(DogID) AS NumDog
FROM (SELECT d.dog_guid AS DogID, d.breed_type, d.dog_fixed AS DogFix,
        CASE 
            WHEN d.breed_type='Pure Breed' THEN 'Pure_Breed'
            ELSE 'Not_Pure_Breed' 
            END AS BreedType,
      
        COUNT(c.created_at) AS NumTests
        FROM dogs d, complete_tests c
        WHERE d.dog_guid=c.dog_guid AND (d.exclude = 0 OR d.exclude is NULL)
        GROUP BY d.dog_guid) AS BreedData
GROUP BY BreedType, DogFix
ORDER BY NumTests DESC;


## 4. Other dog features that might be related to the number of tests completed, and a note about using averages as summary metrics

**Question 17: Adapt your query from Question 7 to include a column with the standard deviation for the number of tests completed by each Dognition personality dimension.**


In [ ]:
%%sql
SELECT dimension AS Dimension, AVG(NumTests) AS NumTests, STDDEV(NumTests) AS STDev_Tests, COUNT(DogID) AS NumDog
FROM (SELECT d.dog_guid AS DogID, d.dimension AS Dimension, COUNT(c.created_at) AS NumTests
        FROM dogs d, complete_tests c
        WHERE d.dog_guid=c.dog_guid AND (d.dimension is not NULL AND d.dimension != '') 
          AND (d.exclude = 0 OR d.exclude is NULL)
        GROUP BY DogID) AS DimensionData
GROUP BY Dimension
ORDER BY NumTests DESC;


**Question 18: Write a query that calculates the average amount of time it took each dog breed_type to complete all of the tests in the exam_answers table. Exclude negative durations from the calculation, and include a column that calculates the standard deviation of durations for each breed_type group:**


In [ ]:
%%sql
SELECT d.breed_type AS BreedType, AVG(TIMESTAMPDIFF(minute, e.start_time, e.end_time)) AS duration,
    STDDEV(TIMESTAMPDIFF(minute, e.start_time, e.end_time)) AS deviation
FROM dogs d, exam_answers e
WHERE d.dog_guid=e.dog_guid AND TIMESTAMPDIFF(minute, e.start_time, e.end_time)>=0
GROUP BY BreedType;